In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## 1 Load Dataset


In [5]:
df = pd.read_csv('./data/temperature_data.csv')
df.head()

,Day,Temperature
0,1,0.133553
1,2,0.061635
2,3,0.175963
3,4,0.302001
4,5,0.082970


##### Goal of generation:

- perhaps look at first 30 - 60 days of temperature data
- then from pattern, generate for next n days, following timeseries forecasting prediction to generate
- i.e not one-step prediction but range-prediction


In [6]:
df.isna().sum()

Day            0
Temperature    0
dtype: int64

In [7]:
temp_data = df['Temperature'].values

### 1.1 Data Preprocessing (Time-series)

- specify custom dataset (prep time-series data for training)
- efficient data handling
- compatible with data loaders
- customized preprocessing
  - converting data to tensors

### Usage steps of Custom Dataset class

1. pass temperature data to custom dataset for sequence to be created
2. create dataloader with help of custom dataset
3. feed dataloader to model


In [17]:
T = 30 # sequence length
batch_size = 32 # No. of sequences to be trained per iteration

# Dataset -> handling, managing data in structured way
class TimeSeriesDataset(Dataset): 
    def __init__(self, data, sequence_length): # init custom dataset
        self.data = data # to hold the temperature data
        self.sequence_length = sequence_length # No. time steps to predict next value
    
    # return total No. of sequence that can be created, given no overlap
    # eg: total data = 3, sequence length = 2, then we have 3 - 2 = 1 (1,2) or (2,3)
    # +1 if overlap allowed
    def __len__(self):  
        return len(self.data) - self.sequence_length

    # retrieve specific sequence & corresponding target value
    # i is start idx
    def __getitem__(self, i):
        x = self.data[i: i+self.sequence_length] # sequence data
        y = self.data[i+self.sequence_length] # target
        
        x = torch.tensor(x, dtype=torch.float32).unsqueeze(-1) # batch_size x sequence_length x input_dim
        y = torch.tensor(y, dtype=torch.float32)

        return x, y


In [18]:
temp_dataset = TimeSeriesDataset(temp_data, T) # T = sequence_length
temp_dataloader = DataLoader(temp_dataset, batch_size=batch_size, shuffle=False) # time-series should not shuffle data, order is impt

## 2 Generator / Discriminator Model Init

- the design of the Neural network would be different for time-series data
- time-series data = LSTM


In [19]:
class Generator(nn.Module):
    def __init__(self, z_dim, hidden_dim, output_dim, seq_len): # z_dim is the input dim
        super(Generator, self).__init__() 
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(z_dim, hidden_dim, num_layers=2, batch_first=True) # batch_first means (batchsize x seq_length x hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim) # adding linear layer + activation after can help increase non-linearity and make model capture more complex r/s
    
    def forward(self, x): # pass data through the layers created
        h_0 = torch.zeros(2, x.size(0), self.hidden_dim) # to store all hidden states for LSTM (2 hidden layers, batchsize, hidden_dim)
        c_0 = torch.zeros(2, x.size(0), self.hidden_dim) # to store all cell states for LSTM
        
        lstm_out, _ = self.lstm(x, (h_0, c_0)) # pass x & hidden + cell states thru layers, _ represents the final hidden and cell state output
        out = self.fc(lstm_out)
        return out

class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim): 
        super(Discriminator, self).__init__() 
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=2, batch_first=True) # batch_first means (batchsize x seq_length x hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1) # output of discriminator is always 1, to tell if synthetic data is real or fake
    
    def forward(self, x): # pass data through the layers created
        h_0 = torch.zeros(2, x.size(0), self.hidden_dim) # to store all hidden states for LSTM (2 hidden layers, batchsize, hidden_dim)
        c_0 = torch.zeros(2, x.size(0), self.hidden_dim) # to store all cell states for LSTM
        
        lstm_out, _ = self.lstm(x, (h_0, c_0)) # _ represents the final hidden and cell state output
        out = self.fc(lstm_out[:, -1,:]) # want (all batchsize, last_sequence, all hidden_dim) | evaluate only final_sequence for decision making of realness
        return out

## 3 Model Init and Training


In [20]:
# Parameters
z_dim = 1 # noise vector dim 1 for temperature
output_dim = 1 # predicting single value temperature
hidden_dim = 64 # no. of nodes in hidden layer for LSTM, more nodes = more complex capture
num_epochs = 1000
lr = 0.0001

# Init Models
generator = Generator(z_dim, hidden_dim, output_dim, seq_len=T)
discriminator = Discriminator(z_dim, hidden_dim)

# Define Loss & Optim
criterion = nn.BCEWithLogitsLoss() # combines BCE with Sigmoid, makes it more numerically stable

g_optim = optim.Adam(generator.parameters(), lr=lr)
d_optim = optim.Adam(discriminator.parameters(), lr=lr)


In [ ]:
# Training loop
for epoch in range(num_epochs):
    
    # Accumulate loss per epoch
    g_loss_epoch = 0
    d_loss_epoch = 0
    
    for real_batch, _ in temp_dataloader: # second element is usually label, but in this 1D temp data, no labels and just placeholder
        batch_size = real_batch.size(0) # [32, 30, 1] - batchsize, sequence length, input dim (temp)
        
        real_labels = torch.ones(batch_size, 1) # real_labels mean probability of 1, create batch_size x 1 col
        fake_labels = torch.zeros(batch_size, 1) # fake_labels means probability of 0
        # tensor([[1], [1]]) for real labels etc.

        # Train Discriminator
        d_optim.zero_grad() # zero gradient in discriminator to prevent accumulation from previous iteration before training
        
        real_output = discriminator(real_batch) # train real output first
        d_loss_real = criterion(real_output, real_labels)
        
        noise = torch.randn(batch_size, T, z_dim) # T - sequence length
        fake_data = generator(noise).detach() # generate fake outputs, detach to prevent gradients propagating thru generator
        fake_output = discriminator(fake_data)
        d_loss_fake = criterion(fake_output, fake_labels)
        
        d_loss_total = d_loss_real + d_loss_fake # want to minimize loss
        d_loss_total.backward() # backprop to calc gradient
        d_optim.step() #grad descent to update weights